classwork: https://colab.research.google.com/drive/1Im7guDVqq5SK0UX6w4bc_ZUsLHyAsDeB

Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы.

In [3]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Сформируем словарь на основе нашего набора текстов. Для этого используем модуль CountVectorizer:

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

n = 854

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True,min_df = 8,max_df = 0.09)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.09, max_features=None, min_df=8,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'however', 'became', 'anywhere', 'an', 'both', 'am', 'anyhow', 'since', 'someone', 'is', 'almost', 'ten', 'as', 'in', 'they', 'hence', 'name', 'much', 'yourself', 'hereby', 'last', 'least', 'thereupon', 'get', 'whereas', 'any', 'move', 'now', 'seems', 'our', 'put', 'can', 'too'...her', 'perhaps', 'whoever', 'bill', 'hers', 'detail', 'herself', 'among', 'into', 'take', 'nobody'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Заведем счетчики n_d_k, n_k_w, n_k из нулей

In [14]:
import random
x,y = X_train.shape
K = 20
len_dict = len(vectorizer.vocabulary_)
Dict = vectorizer.vocabulary_
n_d_k = np.zeros(shape = (x,K))
n_k_w = np.zeros(shape = (K,len_dict))
n_k = np.zeros(shape = K)
columns = X_train.nonzero()[1]
rows = X_train.nonzero()[0]
non_zero_len= X_train.data.shape[0]
print(n_d_k.shape,n_k_w.shape)

(11314, 20) (20, 12424)


Изменим значения  в X_train.data и счетчиков

In [15]:
X_train.data = np.random.randint(low =1 ,high = 21,size = (non_zero_len,))
for i in range(non_zero_len):
    n_k[X_train.data[i]-1] += 1
    n_k_w[X_train.data[i]-1][columns[i]] += 1
    n_d_k[rows[i]][X_train.data[i]-1] += 1

Реализуем алгоритм случайного блуждания

In [19]:
def walk(niter,X_train,n_k,n_k_w,n_d_k,K):
    
    non_zero_len= X_train.data.shape[0]
    columns = X_train.nonzero()[1]
    rows = X_train.nonzero()[0]
    non_zero = X_train.data
    
    a = np.ones(K)
    b = np.ones(X_train.shape[1])
    b_summ = b.sum()
    num = list(np.arange(0,K)+1)
    
    for j in range(niter):
        for i in range(non_zero_len):
                n_k[non_zero[i]-1] -= 1
                n_k_w[non_zero[i]-1][columns[i]] -= 1
                n_d_k[rows[i]][non_zero[i]-1] -= 1
                
                probability = np.array([(n_d_k[rows[i]][c]+a[c])*(n_k_w[c][columns[i]]+b[columns[i]])/(n_k[c]+b_summ)
                                                                                                        for c in range(K)])
                non_zero[i] = int(random.choices(num, weights =  probability/probability.sum())[0])
                
                n_k[non_zero[i]-1] += 1
                n_k_w[non_zero[i]-1][columns[i]] += 1
                n_d_k[rows[i]][non_zero[i]-1] += 1
    return non_zero,n_k,n_k_w,n_d_k

In [21]:
rezult,_,n_k_w,_ = walk(50,X_train,n_k,n_k_w,n_d_k,K)

In [22]:
maximus = np.zeros(shape = (K,10),dtype = int)
for ni in range(10):
    maxis = np.zeros(K,dtype = int)
    for i in range(len_dict):
        maxis = [i if n_k_w[j][i]>n_k_w[j][maxis[j]] else maxis[j] for j in range(K)]
    for j in range(K):
        maximus[j][ni] = maxis[j]
        n_k_w[j][maxis[j]] = 0


In [27]:
for i in range(K):
    dat = np.zeros((1, len_dict))
    for word in maximus[i]:
        dat[0, word] = 1
    print('Theme {}\t\t{}'.format(i,' '.join(vectorizer.inverse_transform(dat)[0])))

Theme 0		cs edu ideas looking posting process remember saying seen sorry
Theme 1		best game games league play players season team win year
Theme 2		couple course different edu far heard luck number tom yes
Theme 3		anybody appreciated edu knows list newsgroup post read sure thing
Theme 4		better didn going got ll maybe said sure thing things
Theme 5		banks cause disease edu gordon intellect medical pitt soon surrender
Theme 6		10 11 12 14 16 17 18 19 25 45
Theme 7		10 current high hot light line low power radio using
Theme 8		case control crime government gun guns law laws rights state
Theme 9		bike buy car cars engine friend looking miles nice road
Theme 10		children history israel israeli jews killed state war world years
Theme 11		advance anybody appreciate edu got hi looking mail post sorry
Theme 12		al bob com dave internet look phone question thank yes
Theme 13		code file files ftp help program set using window windows
Theme 14		believe edu heard interested kind post questions re

Попробуем сделать выводы о содержании глав:

0) Что-то про извинения

1) Тема про командные игры

2) Трудно определить

3) Тема связанная со знаниями и чтением 

4) Трудно определить

5) Что-то с общественными местами

6) Цифры

7) Что-то про радио

8) Про политику  и оружее

9) Про колесный транспорт

10) Про войну в Израиле

11) Трудно определить

12) Трудно определить

13) Написание программ в  Windows

14) Трудно определить

15) Космические исследования 1993 

16) Прослушивание телефонов правительством

17) Программа Клинтона

18) Устройство компьютера

19) Христианство

In [26]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Большинство догадок не совпали